### Performing Web Scraping For A Skin Care Company

## Business Understanding

Veefyed (client) wants to understand the product catalog from AfroGlamour Cosmetics. Right now, the data is unstructured, scattered across web pages, and not available in a format that supports analytics or reporting.

By scraping, cleaning, and structuring this data, Veefyed can:

1. Analyze product features and pricing.

2. Identify common ingredients and their frequency.

3. Better understand which products align with customer skin concerns.

4. Store all images for marketing and catalog purposes.


### Problem Statement

The product information on AfroGlamour Cosmetics’ website is not centralized. It exists only as HTML content and scattered images.

- No single file contains all product details.
- Images are not downloadable in bulk.

- Ingredient lists are long and inconsistent.

This makes it difficult for analysts and business teams to:

- Compare products side by side.

- Create structured reports.

- Identify commonalities (like ingredients for skin concerns).




### Objective

The goal of this project was to collect structured product data from AfroGlamour Cosmetics and organize it into a clean dataset. This included extracting key product details and ensuring that all images associated with each product were downloaded in good quality.


### Stakeholders

- Business/Marketing Team – Needs product data for promotions, price comparisons, and catalog building.

- Data Analysts – Require structured datasets to perform ingredient analysis, trend analysis, and pricing insights.

- Product Development Team – Can analyze ingredients and skin concerns to identify product gaps.

- Customers (Indirect Stakeholders) – Benefit when product data is clear, consistent, and easier to search or recommend.

- Tech/Data Team – Responsible for collecting, cleaning, and structuring the raw product data.




#### Key Features of the Solution

1. Web Scraping Engine
Collects product details (ID, brand, description, price, etc.) from product pages.

2. Image Extraction & Download
Downloads all product images in high quality, stores them in a local folder, and links them in the dataset.

3. Data Structuring (CSV/Excel)
Clean, tabular dataset for easy use in Excel, Power BI, or Python notebooks.

4. Ingredient Grouping Table
Frequency count of ingredients across products to highlight most used ingredients.

5. Scalable Approach
Code can be extended to scrape more products, or even other e-commerce sites, with minimal changes.

1. Importing all the relevant libraries

In [1]:
import os
import re
import time
import uuid
import shutil
from urllib.parse import urljoin, urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import glob


In [2]:
# Config - adjust as needed
BASE = "https://afroglamourcosmetics.com"
SHOP_PAGE = urljoin(BASE, "/shop/")
OUTPUT_CSV = "products.csv"
OUTPUT_XLSX = "products.xlsx"
IMAGES_DIR = "images"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/100 Safari/537.36"
}

# Behavior flags
USE_SELENIUM = False        # set True if pages are JS-driven (not used by default)
DOWNLOAD_IMAGES = True      # set False to skip downloading images (faster)
PARALLEL_DOWNLOADS = True   # use threads for image downloads
N_PRODUCTS = 12             # number of products to scrape
SAVE_INTERVAL = 5           # save partial CSV every N products
MAX_WORKERS = 6             # threads for image downloads

### Session and robust request helper

In [3]:
# 2. Session + safe_request
session = requests.Session()
session.headers.update(HEADERS)

def safe_request(url, session=session, retries=3, backoff=1.0, timeout=20):
    for attempt in range(retries):
        try:
            r = session.get(url, timeout=timeout)
            r.raise_for_status()
            return r
        except Exception as e:
            if attempt == retries - 1:
                print(f"[safe_request] Failed: {url} -> {e}")
                return None
            time.sleep(backoff * (attempt + 1))
    return None


Utility helpers (product id, partial save)

In [4]:
# 3. Helpers
def make_product_id(url):
    path = urlparse(url).path.rstrip("/")
    last = path.split("/")[-1] or str(uuid.uuid4())[:8]
    return re.sub(r'[^A-Za-z0-9_\-]', '_', last)

def save_partial(rows, csv_path="products_partial.csv"):
    try:
        pd.DataFrame(rows).to_csv(csv_path, index=False)
        print(f"[save_partial] saved {csv_path} ({len(rows)} rows)")
    except Exception as e:
        print("[save_partial] failed:", e)


Robust image downloader (streaming + size check)

In [5]:
# 4. Robust image downloader

def download_image(url, save_folder, session=session, retries=4, timeout=25):
    if not url:
        return None
    if isinstance(url, str) and "," in url and " " in url:
        url = url.split(",")[0].strip().split(" ")[0]
    url = urljoin(BASE, url)
    for attempt in range(retries):
        try:
            resp = session.get(url, stream=True, timeout=timeout)
            resp.raise_for_status()
            path = urlparse(url).path
            ext = os.path.splitext(path)[1].split('?')[0] or ".jpg"
            fname = re.sub(r'[^A-Za-z0-9_.-]', '_', os.path.basename(path)) or f"{uuid.uuid4().hex[:8]}{ext}"
            filepath = os.path.join(save_folder, fname)
            with open(filepath, "wb") as out:
                shutil.copyfileobj(resp.raw, out)
            if os.path.getsize(filepath) < 1024:
                os.remove(filepath)
                raise ValueError("Downloaded file too small")
            return filepath
        except Exception as e:
            if attempt == retries - 1:
                print(f"[download_image] failed {url} -> {e}")
                return None
            time.sleep(0.5 * (attempt + 1))
    return None


Parse product page (robust selectors + many image attributes)

In [6]:
# 5. parse_product_page

def parse_product_page(url):
    r = safe_request(url)
    if not r:
        return None

    soup = BeautifulSoup(r.text, "html.parser")
    page_text = soup.get_text(" ", strip=True)

    # product name
    name_tag = soup.find(["h1", "h2"], attrs={"class": re.compile(r".*product.*", re.I)}) or soup.find("h1")
    product_name = name_tag.get_text(strip=True) if name_tag else None

    # price
    price = None
    price_tag = soup.find(string=re.compile(r"AED|USD|£|\$"))
    if price_tag:
        p = re.search(r"([A-Z]{2,3}\s*)?(\d{1,3}(?:[.,]\d{2,})?)", price_tag)
        if p:
            price = p.group(0).strip()

    # sku / barcode
    sku = None
    m = re.search(r"SKU[:\s]*([A-Za-z0-9\-\_]+)", page_text, re.I)
    if m:
        sku = m.group(1).strip()
    else:
        m2 = re.search(r"\b(\d{12,13})\b", page_text)
        if m2:
            sku = m2.group(1)

    # description
    desc = ""
    desc_tag = soup.find(id="tab-description") or soup.find("div", class_=re.compile(r"description", re.I))
    if desc_tag:
        desc = desc_tag.get_text(" ", strip=True)
    else:
        p = soup.find("p")
        desc = p.get_text(" ", strip=True) if p else ""

    # brand / product line
    brand = None
    b_tag = soup.find(string=re.compile(r"Brand|brand|Category:", re.I))
    if b_tag:
        try:
            brand = b_tag.find_next().get_text(strip=True)
        except Exception:
            brand = None
    if not brand:
        bc = soup.find("nav", class_=re.compile(r"breadcrumb", re.I))
        if bc:
            items = [it.get_text(strip=True) for it in bc.find_all("a")]
            if items:
                brand = items[-1]

    # ingredients
    ingredients = None
    ing_tag = soup.find(string=re.compile(r"Ingredients", re.I))
    if ing_tag:
        parent = ing_tag.parent
        nxt = parent.find_next(["p", "ul", "div"])
        if nxt:
            ingredients = nxt.get_text(" ", strip=True)

    # size/volume
    size_volume = None
    msize = re.search(r"(\d+\s?(?:ml|mL|g|gm|oz|litre|l))", page_text, re.I)
    if msize:
        size_volume = msize.group(1)

    # skin concern inference
    lower = (desc or "").lower() + " " + page_text.lower()
    skin_concern = None
    if any(word in lower for word in ["brighten", "lighten", "whiten", "dark spot"]):
        skin_concern = "brightening/dark-spots"
    elif any(word in lower for word in ["moistur", "hydr", "dry"]):
        skin_concern = "moisturizing/dry-skin"
    elif "anti-aging" in lower or "age" in lower:
        skin_concern = "anti-aging"
    elif "acne" in lower or "pimple" in lower:
        skin_concern = "acne"

    # images
    imgs = []
    for img in soup.find_all("img"):
        src = (
            img.get("src")
            or img.get("data-src")
            or img.get("data-lazy-src")
            or img.get("data-srcset")
            or img.get("srcset")
        )
        if not src:
            continue
        if isinstance(src, str) and "," in src:
            src = src.split(",")[0].strip().split(" ")[0]
        src = urljoin(BASE, src)
        if any(x in src for x in ["logo", "sprite", "placeholder", "icon"]):
            continue
        imgs.append(src)
    imgs = list(dict.fromkeys(imgs))

    return {
        "product_name": product_name,
        "price": price,
        "sku": sku,
        "description": desc,
        "brand": brand,
        "ingredients": ingredients,
        "size_volume": size_volume,
        "skin_concern": skin_concern,
        "image_urls": imgs,
        "source_url": url
    }


Get product links from shop pages

In [7]:
# 6. get_product_links_from_shop_pages

def get_product_links_from_shop_pages(required=N_PRODUCTS, start_page=1):
    links = []
    page = start_page
    while len(links) < required:
        shop_url = f"{BASE}/shop/page/{page}/" if page > 1 else SHOP_PAGE
        r = safe_request(shop_url)
        if not r:
            print(f"[get_product_links] failed to load {shop_url}")
            break
        soup = BeautifulSoup(r.text, "html.parser")
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if "/product/" in href:
                full = urljoin(BASE, href)
                if full not in links:
                    links.append(full)
        page += 1
        time.sleep(0.4)
        if page > 60:
            break
    return links[:required]


Download images for a product (parallel)

In [8]:
# 7. download_images_for_product
def download_images_for_product(image_urls, pid, parallel=PARALLEL_DOWNLOADS, max_workers=MAX_WORKERS):
    folder = os.path.join(IMAGES_DIR, pid)
    os.makedirs(folder, exist_ok=True)
    image_paths = []
    if not image_urls:
        return []
    if parallel and len(image_urls) > 1:
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            futures = {ex.submit(download_image, url, folder): url for url in image_urls}
            for fut in as_completed(futures):
                res = fut.result()
                if res:
                    image_paths.append(res)
    else:
        for url in image_urls:
            p = download_image(url, folder)
            if p:
                image_paths.append(p)
            time.sleep(0.05)
    return image_paths


Main orchestration: scrape, download, save products.csv/xlsx

In [9]:
# 8. main()
def main(n_products=N_PRODUCTS, download_images=DOWNLOAD_IMAGES):
    os.makedirs(IMAGES_DIR, exist_ok=True)
    product_links = get_product_links_from_shop_pages(required=n_products)
    print(f"[main] Found {len(product_links)} product links. Sample: {product_links[:3]}")
    rows = []
    for idx, url in enumerate(tqdm(product_links, desc="Scraping products"), start=1):
        rec = parse_product_page(url)
        if not rec:
            print(f"[main] skip {url}")
            continue
        pid = make_product_id(url)
        rec["product_id"] = pid
        image_files = []
        if download_images and rec.get("image_urls"):
            try:
                image_files = download_images_for_product(rec["image_urls"], pid)
            except Exception as e:
                print(f"[main] image download error for {pid}: {e}")
        rec["image_files"] = "|".join(image_files)
        row = {
            "product_id": rec["product_id"],
            "product_line_name": rec.get("brand") or "",
            "brand_name": rec.get("brand") or "",
            "product_name": rec.get("product_name") or "",
            "product_description": rec.get("description") or "",
            "product_images": "|".join(rec.get("image_urls", [])),
            "barcode": rec.get("sku") or "",
            "price": rec.get("price") or "",
            "size_volume": rec.get("size_volume") or "",
            "ingredients": rec.get("ingredients") or "",
            "skin_concern": rec.get("skin_concern") or "",
            "image_files": rec.get("image_files") or "",
            "source_url": rec.get("source_url") or ""
        }
        rows.append(row)
        if idx % SAVE_INTERVAL == 0:
            save_partial(rows, csv_path="products_partial.csv")
    df = pd.DataFrame(rows)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"[main] saved CSV => {OUTPUT_CSV}")
    try:
        df.to_excel(OUTPUT_XLSX, index=False)
        print(f"[main] saved Excel => {OUTPUT_XLSX}")
    except Exception as e:
        print(f"[main] could not save Excel ({e}). CSV saved instead.")
    return df


Run the scraper

In [10]:
# 9. Run scraper
if __name__ == "__main__":
    df_products = main(n_products=N_PRODUCTS, download_images=DOWNLOAD_IMAGES)


[main] Found 12 product links. Sample: ['https://afroglamourcosmetics.com/product/lollis-beauty-make-up-professional-touch-smooth-primer-base-make-up-studio-finish-lp-155-2/', 'https://afroglamourcosmetics.com/product/lollis-beauty-make-up-real-look-mascara-with-keratin-lp-252-2/', 'https://afroglamourcosmetics.com/product/lollis-beauty-make-up-long-lasting-tattoo-waterproof-dipliner-lp-300-2/']


Scraping products:   0%|          | 0/12 [00:00<?, ?it/s]

[download_image] failed data:image/svg+xml -> No connection adapters were found for 'data:image/svg+xml'


Scraping products:   8%|▊         | 1/12 [00:10<02:00, 10.99s/it]

[download_image] failed https://www.facebook.com/tr?id=306595973993558&ev=PageView&noscript=1 -> Downloaded file too small
[download_image] failed data:image/svg+xml -> No connection adapters were found for 'data:image/svg+xml'
[download_image] failed https://afroglamourcosmetics.com/wp-content/uploads/2020/11/Clear-Essence-Platinum-Medicated-Fade-Creme-with-Sunscreen-4-oz..jpg -> 403 Client Error: Forbidden for url: https://afroglamourcosmetics.com/wp-content/uploads/2020/11/Clear-Essence-Platinum-Medicated-Fade-Creme-with-Sunscreen-4-oz..jpg


...

Scraping products:  92%|█████████▏| 11/12 [01:58<00:11, 11.35s/it]

[download_image] failed https://afroglamourcosmetics.com/wp-content/uploads/2020/07/Bio-Claire-Lightening-SetCream-Lotion-Serum-Soap-and-oil-600x593.jpeg -> [Errno 2] No such file or directory: 'images\\palmolive-aroma-sensations-feel-good-oil-sensation-with-essential-oils-bright-shower-gel-500ml-2\\Bio-Claire-Lightening-SetCream-Lotion-Serum-Soap-and-oil-600x593.jpeg'
[download_image] failed data:image/svg+xml -> No connection adapters were found for 'data:image/svg+xml'
[download_image] failed https://afroglamourcosmetics.com/wp-content/uploads/2020/11/Clear-Essence-Platinum-Medicated-Fade-Creme-with-Sunscreen-4-oz..jpg -> 403 Client Error: Forbidden for url: https://afroglamourcosmetics.com/wp-content/uploads/2020/11/Clear-Essence-Platinum-Medicated-Fade-Creme-with-Sunscreen-4-oz..jpg


Scraping products: 100%|██████████| 12/12 [02:09<00:00, 10.81s/it]

[download_image] failed https://www.facebook.com/tr?id=306595973993558&ev=PageView&noscript=1 -> Downloaded file too small
[main] saved CSV => products.csv


[main] saved Excel => products.xlsx


Verification (images exist and are referenced)

In [11]:
# 10. Verification
import glob
if not os.path.exists(OUTPUT_CSV):
    print("CSV not found:", OUTPUT_CSV)
else:
    df = pd.read_csv(OUTPUT_CSV)
    print("Rows in CSV:", len(df))
    df['has_image_files'] = df['image_files'].notna() & (df['image_files'].str.len() > 0)
    print("Rows with image_files:", int(df['has_image_files'].sum()))
    sample = df[df['has_image_files']].head(5)
    for _, r in sample.iterrows():
        pid = r['product_id']
        files = r['image_files'].split("|") if r['image_files'] else []
        print(f"{pid} -> {len(files)} images")
        for f in files[:3]:
            exists = os.path.exists(f)
            size = os.path.getsize(f) if exists else "N/A"
            print("   ", f, "exists?", exists, "size:", size)
    image_files_on_disk = list(glob.glob(os.path.join(IMAGES_DIR, "**", "*.*"), recursive=True))
    print("Total image files on disk:", len(image_files_on_disk))


Rows in CSV: 12
Rows with image_files: 12
lollis-beauty-make-up-professional-touch-smooth-primer-base-make-up-studio-finish-lp-155-2 -> 49 images
    images\lollis-beauty-make-up-professional-touch-smooth-primer-base-make-up-studio-finish-lp-155-2\glamour.png exists? True size: 4754
    images\lollis-beauty-make-up-professional-touch-smooth-primer-base-make-up-studio-finish-lp-155-2\Perfume-Bundle-Afro-Glamour-Cosmetics.png exists? True size: 410679
    images\lollis-beauty-make-up-professional-touch-smooth-primer-base-make-up-studio-finish-lp-155-2\wellbeingproducts.jpg exists? True size: 80680
lollis-beauty-make-up-real-look-mascara-with-keratin-lp-252-2 -> 48 images
    images\lollis-beauty-make-up-real-look-mascara-with-keratin-lp-252-2\glamour.png exists? True size: 4754
    images\lollis-beauty-make-up-real-look-mascara-with-keratin-lp-252-2\facial-Care.jpg exists? True size: 14827
    images\lollis-beauty-make-up-real-look-mascara-with-keratin-lp-252-2\hair-care.jpg exists? True

Grouped ingredients: clean, explode, counts, mapping, save

In [12]:
# 11. Grouped ingredient table (runs after scraping)

if not os.path.exists(OUTPUT_CSV):
    raise FileNotFoundError(f"{OUTPUT_CSV} not found. Run scraper first.")
df = pd.read_csv(OUTPUT_CSV)

# helper to clean ingredient text
def clean_ingredients_text(txt):
    if pd.isna(txt) or str(txt).strip()=="":
        return ""
    s = str(txt)
    s = re.sub(r'[\r\n]', ' ', s)
    s = s.replace(';', ',').replace('|', ',').replace('/', ',')
    s = re.sub(r'\([^)]*\)', '', s)
    s = re.sub(r'\s*,\s*', ',', s)
    s = re.sub(r'\s+', ' ', s)
    s = s.strip().lower()
    s = re.sub(r',+$', '', s)
    return s

df['ingredients_clean'] = df['ingredients'].fillna("").apply(clean_ingredients_text)

def split_ingredients(txt):
    if not txt:
        return []
    parts = [p.strip() for p in txt.split(',') if p.strip()]
    expanded = []
    for p in parts:
        for sub in re.split(r'\sand\s|\s\+\s', p):
            sub = sub.strip()
            if sub:
                expanded.append(sub)
    return expanded

df['ingredients_list'] = df['ingredients_clean'].apply(split_ingredients)
df_exploded = df[['product_id','product_name','ingredients_list']].explode('ingredients_list').rename(columns={'ingredients_list':'ingredient'})
df_exploded['ingredient'] = df_exploded['ingredient'].astype(str).str.strip()
df_exploded = df_exploded[df_exploded['ingredient'] != ""].reset_index(drop=True)

# simple normalization map — extend as needed
norm_map = {
    'aqua': 'water',
    'water': 'water',
    'glycerin': 'glycerin'
    # add more mappings after inspection
}

def normalize_ingredient(ing):
    ing0 = ing.strip().lower()
    ing0 = re.sub(r'[^a-z0-9\s\-]', '', ing0)
    ing0 = re.sub(r'\s{2,}', ' ', ing0)
    if ing0 in norm_map:
        return norm_map[ing0]
    return ing0

df_exploded['ingredient_norm'] = df_exploded['ingredient'].apply(normalize_ingredient)

# counts and mapping
ingredient_counts = (
    df_exploded.groupby('ingredient_norm')['product_id']
    .nunique()
    .reset_index()
    .rename(columns={'product_id':'product_count'})
    .sort_values('product_count', ascending=False)
)

ingredient_product_map = (
    df_exploded.groupby('ingredient_norm')
    .agg({
        'product_id': lambda ids: "|".join(sorted(set(ids))),
        'product_name': lambda names: "|".join(sorted(set(names)))
    })
    .reset_index()
    .rename(columns={'product_id':'product_ids','product_name':'product_names'})
)

ingredient_counts.to_csv("ingredient_counts.csv", index=False)
ingredient_product_map.to_csv("ingredient_product_map.csv", index=False)

# save both sheets to Excel if possible
try:
    with pd.ExcelWriter("ingredient_summary.xlsx", engine="openpyxl", mode="w") as writer:
        ingredient_counts.to_excel(writer, sheet_name="counts", index=False)
        ingredient_product_map.to_excel(writer, sheet_name="product_map", index=False)
    print("Saved ingredient_summary.xlsx")
except Exception as e:
    print("Could not write Excel (engine missing?). CSVs saved instead.", e)

print("Saved: ingredient_counts.csv, ingredient_product_map.csv")


Saved ingredient_summary.xlsx
Saved: ingredient_counts.csv, ingredient_product_map.csv


Zip images folder and final checklist

In [13]:
# 12. Zip images and show final deliverables

if os.path.exists(IMAGES_DIR) and any(os.scandir(IMAGES_DIR)):
    shutil.make_archive("images_archive", 'zip', IMAGES_DIR)
    print("Created images_archive.zip")
else:
    print("No images found to zip in", IMAGES_DIR)

print("\nDeliverables produced:")
print("- products.csv")
print("- products.xlsx (if engine installed)")
print("- images/ (downloaded images by product_id)")
print("- images_archive.zip")
print("- ingredient_counts.csv")
print("- ingredient_product_map.csv")
print("- ingredient_summary.xlsx (if engine installed)")
print("- products_partial.csv (interim, optional)")


Created images_archive.zip

Deliverables produced:
- products.csv
- products.xlsx (if engine installed)
- images/ (downloaded images by product_id)
- images_archive.zip
- ingredient_counts.csv
- ingredient_product_map.csv
- ingredient_summary.xlsx (if engine installed)
- products_partial.csv (interim, optional)
